<a href="https://colab.research.google.com/github/parthag1201/Authentication_google/blob/main/MAF_Customer_Support_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Env and Agent setup

In [ ]:
### General environment setup

# Install packages for crew and langchain
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_google_genai

import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# Warning Control
import warnings
warnings.filterwarnings('ignore')

# Import crewAI library
from crewai import Agent, Task, Crew

import os
from google.colab import userdata # For API Secret
# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

os.environ["GEMINI_MODEL_NAME"] = 'gemini-2.5-flash'

In [ ]:
from langchain_community.llms import HuggingFaceHub # Import necessary class if using Hugging Face models
from langchain_google_genai import ChatGoogleGenerativeAI # Import the correct class for Gemini

# Initialize the Gemini LLM
os.environ['GOOGLE_API_KEY']=userdata.get('Gemini_ai_api')
gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [ ]:
# Agents

support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
  llm = gemini_llm
)

support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
  llm = gemini_llm
)

# Tools, Guardrials and Memory

In [ ]:
# SerperDevTool - External service to scrape google
# ScrapeWebsiteTool - Scraper for agent to access URL and extract contents of the webpage
# WebsiteSearchTool - RAG over a website (Semantic search)
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

# Possible Custom Tools
"""
Load customer data
Tap into previous conversations
Load data from a CRM
Checking existing bug reports
Checking existing feature requests
Checking ongoing tickets
"""

' \nLoad customer data\nTap into previous conversations\nLoad data from a CRM\nChecking existing bug reports\nChecking existing feature requests\nChecking ongoing tickets\n'

In [ ]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [ ]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

# web_search_tool = SerperDevTool(
#     website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
# )

In [ ]:
# print(docs_scrape_tool)

In [ ]:
# Tasks

inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	  tools=[docs_scrape_tool],
    agent=support_agent,
)

quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [ ]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
)

In [ ]:
inputs = {
    "customer": "Amazon.com",
    "person": "Jeff Bezos",
    "inquiry": "I need help with setting up a Crew to help me automate Software developemnt in Amazon "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)

Hello Mr. Bezos,

It's an absolute pleasure to assist Amazon.com with your CrewAI implementation! We're thrilled to help you automate software development with our framework. I'm here to provide you with the most comprehensive guidance based on the available documentation for setting up and understanding your Crew, and addressing your specific question about memory.

Let's break down how to set up a Crew and how it works, and then we'll discuss memory and how to kick off your amazing Crew!

### Setting Up a CrewAI Crew for Software Development Automation

CrewAI is a powerful, lean, and lightning-fast Python framework designed to help you create autonomous AI agent teams that work together to tackle complex tasks, much like departments within a company collaborate. For automating software development, you'd leverage these core components:

1.  **Crew (The Orchestrator):** This is the top-level organization within CrewAI. Think of it as the project manager for your software development automation. The Crew manages your AI agent teams, oversees the overall workflows, ensures seamless collaboration among agents, and ultimately delivers the desired outcomes (e.g., code, documentation, test plans).

2.  **AI Agents (The Specialized Team Members):** These are the individual AI workers in your software development team. Each agent has a specific role, expertise, and goals. For instance, you could have:
    *   A **"Software Architect"** agent (role) responsible for high-level design.
    *   A **"Code Developer"** agent (role) focused on writing code.
    *   A **"Quality Assurance Engineer"** agent (role) for testing and debugging.
    *   A **"Documentation Specialist"** agent (role) for creating project documentation.
    Each agent can be equipped with specific tools (e.g., code interpreters, access to version control systems, bug trackers, documentation generators) to perform their tasks effectively and make autonomous decisions within their defined scope.

3.  **Tasks (The Assignments):** These are the individual assignments given to your agents. Each task should have clear objectives and specify the tools an agent might need to complete it. For example:
    *   A task for the "Software Architect" might be: "Design the microservices architecture for the new e-commerce feature."
    *   A task for the "Code Developer" might be: "Implement the user authentication module following the architecture design."
    *   A task for the "Quality Assurance Engineer" might be: "Write and execute unit tests for the user authentication module."
    Tasks feed into the larger process, contributing to the overall software development goal.

4.  **Process (The Workflow Management System):** This defines how your agents collaborate and how tasks are assigned and managed. It ensures efficient execution and interaction between your agents. CrewAI supports defining sequential or parallel workflows, allowing agents to automatically handle task dependencies. For example, the "Code Developer" task might depend on the "Software Architect" task being completed.

**How It All Works Together:**
In a software development automation scenario:
*   The **Crew** organizes the entire operation.
*   **AI Agents** work on their specialized **Tasks**.
*   The **Process** ensures smooth collaboration and efficient flow of work, from design to development to testing.
*   All **Tasks** get completed collaboratively to achieve the overarching software development goal, like delivering a new feature or fixing a bug.

### Kicking Off Your Crew

While the documentation I accessed provides a conceptual overview of how Crews work and how their components interact, we've gone ahead and pulled the exact programmatic command you'll need to get your Crew running! The "kick-off" is the moment when the Crew begins its operation, with agents working on their tasks according to the defined process to achieve the goal.

The command to initiate and run a CrewAI Crew, triggering its execution of tasks, is the `kickoff()` method. This method starts the entire process, allowing the agents to collaborate and complete their assigned tasks.

**Official Documentation Reference:** This method is fundamental and is demonstrated across various foundational examples, including the "Build Your First Crew Step-by-step tutorial" and the general "Getting Started" guides.

**Precise Code Example:**

```python
from crewai import Crew, Process
from crewai import Agent, Task # Assuming Agent and Task are defined elsewhere

# --- Define your Agents ---
# Example Agent (replace with your actual agent definitions)
researcher = Agent(
    role='Senior Research Analyst',
    goal='Uncover groundbreaking insights from latest AI advancements',
    backstory='A seasoned analyst with a knack for identifying emerging trends.',
    verbose=True,
    allow_delegation=False
)

# --- Define your Tasks ---
# Example Task (replace with your actual task definitions)
research_task = Task(
    description='Analyze the latest advancements in large language models and identify potential applications in e-commerce.',
    expected_output='A concise report detailing key LLM advancements and 3-5 concrete e-commerce application ideas.',
    agent=researcher
)

# --- Instantiate your Crew ---
# Create a Crew with your defined agents and tasks
my_crew = Crew(
    agents=[researcher], # Add all your agents here
    tasks=[research_task], # Add all your tasks here
    process=Process.sequential, # Or Process.hierarchical
    verbose=2 # Optional: Set to 1 for simple output, 2 for detailed output
)

# --- Kick Off the Crew ---
# This is the command that starts the crew's execution
print("## Crew starting to work...")
result = my_crew.kickoff()

# --- Access the result ---
print("\n########################")
print("## Crew finished working!")
print("########################\n")
print(result) # The 'result' variable will contain the final output of the crew.
```

**Detailed Instructions:**
1.  **Define your Agents and Tasks:** Before you can kick off a crew, you must first define your `Agent` and `Task` instances. Each agent should have a `role`, `goal`, and `backstory`. Each task needs a `description`, `expected_output`, and an assigned `agent`.
2.  **Instantiate the `Crew`:** Create an instance of the `Crew` class, passing your list of `agents` and `tasks`. You also need to specify the `process` (e.g., `Process.sequential` or `Process.hierarchical`) and can optionally set `verbose` for more detailed logging.
3.  **Call `kickoff()`:** Once your `Crew` object is instantiated, simply call the `.kickoff()` method on that object. This method orchestrates the execution flow, delegating tasks to agents as per the defined process.
4.  **Capture the Output:** The `kickoff()` method returns the final output of the crew's execution, which can be captured in a variable (e.g., `result`) for further processing or display.

### Adding Memory to Your Crew

Regarding your question about adding memory to your Crew, this is a fantastic feature for enabling agents to retain context and learn from past interactions, which is crucial for complex, long-running software development projects.

Adding memory to a CrewAI Crew is crucial for enabling agents to retain context, learn from past interactions, and ensure continuity across tasks, especially in multi-step or long-running processes. This is achieved by setting a specific parameter during the Crew initialization.

**Official Documentation Reference:** The dedicated "Memory" section of our documentation details how to enable and utilize this feature.

**Specific Method and Parameters:**
Memory is enabled by setting the `memory` parameter to `True` when initializing the `Crew` object.

**Precise Code Example:**

```python
from crewai import Crew, Process, Agent, Task

# --- Define your Agents ---
# Example Agent 1
researcher_agent = Agent(
    role='Senior Market Researcher',
    goal='Analyze market trends and customer feedback for new product ideas.',
    backstory='An expert in market analysis, skilled at synthesizing data into actionable insights.',
    verbose=True,
    allow_delegation=False
)

# Example Agent 2
product_designer_agent = Agent(
    role='Innovative Product Designer',
    goal='Translate research insights into tangible product concepts.',
    backstory='A creative designer with a flair for user-centric solutions.',
    verbose=True,
    allow_delegation=True # Allows delegating parts of design if needed
)

# --- Define your Tasks ---
# Task 1: Market Research
market_research_task = Task(
    description='Conduct a comprehensive analysis of the current e-commerce market for smart home devices, identifying key trends, competitor offerings, and customer pain points.',
    expected_output='A detailed market research report highlighting opportunities for a new smart home product.',
    agent=researcher_agent
)

# Task 2: Product Concept Development
product_concept_task = Task(
    description='Based on the market research report, develop 3-5 innovative product concepts for a new smart home device, including their core features and unique selling propositions.',
    expected_output='A presentation slide deck outlining 3-5 distinct smart home product concepts with feature sets.',
    agent=product_designer_agent
)

# --- Instantiate your Crew with Memory Enabled ---
# The key part for enabling memory is setting 'memory=True'
crew_with_memory = Crew(
    agents=[researcher_agent, product_designer_agent],
    tasks=[market_research_task, product_concept_task],
    process=Process.sequential, # Tasks will run one after another
    memory=True,  # <--- This parameter enables memory for the entire crew
    verbose=2     # Optional: Set to 1 for simple output, 2 for detailed output
)

# --- Kick Off the Crew ---
print("## Crew with Memory starting to work...")
result_with_memory = crew_with_memory.kickoff()

print("\n########################")
print("## Crew with Memory finished working!")
print("########################\n")
print(result_with_memory)
```

**Detailed Instructions:**
1.  **Ensure Crew Initialization:** Memory is a property of the `Crew` itself, not individual agents or tasks. Therefore, you must enable it at the time of the `Crew` object's instantiation.
2.  **Set `memory=True`:** When creating your `Crew` instance, simply add `memory=True` to its parameters.
    *   `crew = Crew(agents=[...], tasks=[...], process=Process.sequential, memory=True, verbose=2)`
3.  **How Memory Works:** When `memory=True` is set, the CrewAI framework will automatically manage and pass relevant context (e.g., outputs from previous tasks, key information from agent interactions) to subsequent agents and tasks. This ensures that agents have a more comprehensive understanding of the ongoing process and don't "forget" prior information, leading to more coherent and intelligent multi-step executions.
4.  **No Additional Code for Memory Management (Typically):** You generally do not need to write explicit code to "add" or "retrieve" items from memory once `memory=True` is set. The framework handles this internally, providing the context to the agents as needed for their next actions. The agents' prompts are dynamically updated with relevant past interactions.

---

Mr. Bezos, we've gone the extra mile to ensure you have the most precise and actionable guidance directly from our documentation. We are fully committed to ensuring Amazon.com has the best experience with CrewAI, and I am here to assist you every step of the way. Please let me know if there's anything else I can help with!

Best regards,

Your Senior Support Representative at crewAI